<a href="https://colab.research.google.com/github/qw-4735/PyTorch/blob/main/%EB%AC%B8%EC%9E%90_%EB%8B%A8%EC%9C%84_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 훈련 데이터 전처리하기

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [4]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str + label_str)))  # 현재 문자 집합에는 총 5개의 문자가 있다 (!,a,e,l,p)
vocab_size = len(char_vocab)
print('문자 집합의 크기 : {}'.format(vocab_size))   

문자 집합의 크기 : 5


In [5]:
input_size = vocab_size   # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [7]:
char_to_index = dict((c,i) for i,c in enumerate(char_vocab))   # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [8]:
index_to_char = {}    
for key, value in char_to_index.items() :
  index_to_char[value] = key
print(index_to_char) 

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [9]:
x_data = [char_to_index[c] for c in input_str]  # input_str = 'apple'
y_data = [char_to_index[c] for c in label_str]  # label_str = 'pple!'
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [10]:
# 배치 차원 추가
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [11]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [12]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  """Entry point for launching an IPython kernel.


In [13]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


### 2. RNN 모델 구현하기

In [14]:
class Net(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size) :
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)  # RNN 셀 구현
    self.fc = torch.nn.Linear(hidden_size, output_size, bias=True)  # 출력층 구현
  def forward(self, x):  # 구현한 RNN 셀과 출력층을 연결
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x


In [15]:
net = Net(input_size, hidden_size, output_size)

In [16]:
outputs= net(X)
print(outputs.shape)   # 3차원 텐서

torch.Size([1, 5, 5])


In [17]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [18]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.6541461944580078 prediction:  [[1 1 1 1 1]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  aaaaa
1 loss:  1.3844155073165894 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
2 loss:  1.2018929719924927 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
3 loss:  0.9639561772346497 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
4 loss:  0.7215503454208374 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
5 loss:  0.5170650482177734 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.3501472473144531 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.2560461163520813 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.1752525418996811 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.11990358680486679 prediction:  [[4 4 3 2 0]]